In [1]:
import torch

import meshzoo

from torchfem import Solid
from torchfem.materials import Isotropic

# Material model

Isotropic linear elastic stiffness tensor

In [2]:
# Material model
material = Isotropic(E=1000.0, nu=0.3)

# 3D Cube with hexaeders

In [3]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1

box = Solid(nodes, elements, forces, displacements, constraints, material.C())

In [4]:
u, f = box.solve()

In [5]:
box.plot(u=u, node_property={"Disp": u})

Widget(value="<iframe src='http://localhost:63378/index.html?ui=P_0x2a1d9d960_0&reconnect=auto' style='width: …

# 3D Cube with hexaeders and inelastic (e.g. thermal) strain

In [6]:
# Reset constraints
displacements = torch.zeros_like(nodes)
constraints[:, :] = False
constraints[nodes[:, 0] == 0.0, 0] = True
constraints[nodes[:, 0] == 1.0, 0] = True
constraints[nodes[:, 1] == 0.5, 1] = True
constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
strains = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

box = Solid(nodes, elements, forces, displacements, constraints, material.C(), strains)

In [7]:
u, f = box.solve()
box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value="<iframe src='http://localhost:63378/index.html?ui=P_0x16fd5df30_1&reconnect=auto' style='width: …

In [8]:
sigma = box.compute_stress(u)
print(f"Mean stress in x is {sigma[:,0].mean()}. It should be {-material.E() * s}.")

Mean stress in x is 100.0. It should be 100.0.


# 3D Cube with tetrahedrons

In [9]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1


box = Solid(nodes, elements, forces, displacements, constraints, material.C())

In [10]:
u, f = box.solve()

In [11]:
box.plot(u=u, node_property={"Disp": u})

Widget(value="<iframe src='http://localhost:63378/index.html?ui=P_0x2a484b700_2&reconnect=auto' style='width: …

# 3D Cube with tetrahedrons and inelastic (e.g. thermal) strain

In [12]:
# Reset constraints
displacements = torch.zeros_like(nodes)
constraints[:, :] = False
constraints[nodes[:, 0] == 0.0, 0] = True
constraints[nodes[:, 0] == 1.0, 0] = True
constraints[nodes[:, 1] == 0.5, 1] = True
constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
strains = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

box = Solid(nodes, elements, forces, displacements, constraints, material.C(), strains)

In [13]:
u, f = box.solve()
box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value="<iframe src='http://localhost:63378/index.html?ui=P_0x2a4849660_3&reconnect=auto' style='width: …

In [14]:
sigma = box.compute_stress(u)
print(f"Mean stress in x is {sigma[:,0].mean()}. It should be {-material.E() * s}.")

Mean stress in x is 100.0. It should be 100.0.
